<h1 align="center">Python performance exercises</h1>

## Python best practices exercises

### Exercise 1

considering the following function for concatenating list strings with delimiter.

In [1]:
!pip install line_profiler

In [2]:
def ft_concatenate(l_strings, d):
    """concatenate list of strings into one string separated by delimiter"""
    res = l_strings[0]
    for e in l_strings[1:]:
        res = res + d + e
    return res

In [3]:
from random import choice
from string import ascii_lowercase, digits

chars = ascii_lowercase + digits

lst = [''.join(choice(chars) for _ in range(2)) for _ in range(10000)]


In [4]:
%load_ext line_profiler

In [5]:
%timeit ft_concatenate(lst,",")

23.3 ms ± 6.22 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [6]:
%lprun -f ft_concatenate ft_concatenate(lst,",")

As we notice the operation concatination represent the bottlenecks because we read in each step all chars already concataneted so we read first string n time and the second n-1...
the total is nl1+(n-1)l2....ln (l represent the len of the string)

- profile the function and identify the bottlenecks.
- improve speed up of the function
*Hint: you may need to look to the string functions in python documentation*

In [7]:
# write your code here
def ft_concatenate_fast(l_strings, d):
    """concatenate list of strings into one string separated by delimiter"""
    return d.join(l_strings)

In [8]:
%lprun -f ft_concatenate_fast ft_concatenate_fast(lst,",")

### Exercise 2

In this exercise you will solve the following problem using two methods bruteforce method, and fast method.

**Problem:** You are given a list of n integers, and your task is to calculate the number of distinct values in the list.

**Example**
- Input:
5
2 3 2 2 3

- Output:
3

**Implement the following methods:**

1. **bruteforce method:** create an empty list and start adding items for the given list without adding the previous item add, at the end the result list will contain unique values, print lenght of the list and you are done. 
2. **fast method** think of using Set data structure.

- time the two methods, what do you think?

In [9]:
# bruteforce method
def get_distinct_bf(l):
    dis=[]
    for i in l:
        if i not in dis:
            dis.append(i)
    return len(dis)

In [10]:
# fast method
def get_distinct_fast(l):
    return len(set(l))

In [11]:
# Create a random list of numbers for testing
import numpy as np 
l=np.random.choice((1,3000),1000)

# time the two methods
%timeit get_distinct_fast(l)


177 µs ± 22 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [12]:
%timeit get_distinct_bf(l)

309 µs ± 110 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


## Cython exercises

### Exercise 1

1. load the cython extension.

In [11]:
%load_ext cython

2. Considering the following polynomial function:

In [14]:
def poly(a,b):
    return 10.5 * a + 3 * (b**2)

- Create an equivalent Cython function of `poly` with name `poly_cy`.

In [15]:
%%cython 
def poly_cy(double a, double b):
    return 10.5 * a + 3 * (b**2)
print(poly_cy(1.9,3))

46.95


3. time the performance of Python and Cython version of the function, what is the factor of speed up between the two verions.

In [16]:
# write your code here
%timeit poly(20, 2)
%timeit poly_cy(20, 2)

997 ns ± 207 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
449 ns ± 126 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


4. Now let's work on another example using loop.
    - rewrite the same function below fib that calculates the fibonacci sequence using cython, but now try to add type for the variables used inside it, add a prefix `_cy` to your new cython function.

In [17]:
def fib(n):
    a, b = 1, 1
    for i in range(n):
        a, b = a + b, a

    return a

In [18]:
%%cython -a
cpdef fib_cy(int n):
    cdef int i, a, b
    a, b = 1, 1
    for i in range(n):
        a, b = a + b, a
    return a

- time the two function for fibonacci series, with n = 20, what is the factor of speed now, What do you think?

In [19]:
# write your code here
%timeit fib(20)
%timeit fib_cy(20)

3.58 µs ± 565 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
179 ns ± 26.8 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


We acheive *20 speed using cython

5. Recursive functions are functions that call themselves during their execution. Another interesting property of the Fibonacci sequence is that it can be written as a recursive function. That’s because each item depends on the values of other items (namely item n-1 and item n-2)

- Rewrite the fib function using recursion. Is it faster than the non-recursive version? Does Cythonizing it give even more of an advantage? 

In [20]:
%%cython
cpdef fib_rec(int n):
    if n <= 1:
        return n
    else:
        return fib_rec(n-1) + fib_rec(n-2)

In [21]:
%timeit fib_rec(20)

350 µs ± 6.62 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


We remarque despite using cython with recursive fibonacci is less speed in comparaison with the other method because we are calling F(18) twice, F(17) 3 times, F(16) 5 times, F(15) 7 times.. and so on

### Exercise 2

- Monte Carlo methods are a broad class of computational algorithms that rely on repeated random sampling to obtain numerical results. 
- One of the basic examples of getting started with the Monte Carlo algorithm is the estimation of Pi.

**Estimation of Pi**

- The idea is to simulate random (x, y) points in a 2-D plane with domain as a square of side 1 unit. 
- Imagine a circle inside the same domain with same diameter and inscribed into the square. 
- We then calculate the ratio of number points that lied inside the circle and total number of generated points. 
- Refer to the image below:

![demo](../data/MonteCarloPlot.png)

We know that area of the square is 1 unit sq while that of circle is $\pi \ast  (\frac{1}{2})^{2} = \frac{\pi}{4}$. Now for a very large number of generated points,

![demo](../data/MonteCarloCalc.png)


## The Algorithm

1. Initialize cile_points, square_points and interval to 0.
2. Generate random point x.
3. Generate random point y.
4. Calculate d = x*x + y*y.
5. If d <= 1, increment circle_points.
6. Increment square_points.
7. Increment interval.
8. If increment < NO_OF_ITERATIONS, repeat from 2.
9. Calculate pi = 4*(circle_points/square_points).
10. Terminate.

**Your mission:** time the function `monte_carlo_pi`, identify the bottlenecks and create a new version using cython functionality to speed up monte carlo simulation for PI, use 100,000 points and compare the speed up factor between python and cython, considering the following optimizations:
- add type for variables used.
- add type for the function
- use c rand function instead of python rand function.
 
*Hint: you can import function from C libraries using the following approach `from libc.<name of c library> cimport <library function name>`, replace the holders `<>` with the right identities for the current problem*

In [9]:
import random

def monte_carlo_pi(nsamples):
    pi = 0.
    
   # Implement your code here
    cile_points=0
    square_points=0 
    for i in range(nsamples):
        x = random.random()
        y = random.random()
        d = x**2 + y**2
        if d<=1:
            cile_points += 1
    pi=4*(cile_points/nsamples)
    return pi

In [12]:
%%cython
import random
from libc.stdlib cimport rand
from libc.stdlib cimport RAND_MAX
def monte_carlo_pi_cy(int nsamples):
    
    cdef float pi = 0
    cdef int cile_points=0
    cdef int square_points=0 
    cdef float x, y, d
    
    for i in range(nsamples):
        x = rand() / RAND_MAX
        y = rand() / RAND_MAX
        d = x**2 + y**2
        if d<=1:
            cile_points += 1
    pi=4*(cile_points/nsamples)
    return pi


In [63]:
print(monte_carlo_pi(100000))
print(monte_carlo_pi_cy(100000))

3.141
3.1388800144195557


In [64]:
%timeit monte_carlo_pi(100000)
%timeit monte_carlo_pi_cy(100000)

173 ms ± 23.3 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
22.3 ms ± 3.51 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


## Numba exercises

### Exercise 1

Previously we considered how to approximateby Monte Carlo.

- Use the same idea here, but make the code efficient using Numba.
- Compare speed with and without Numba when the sample size is large.

In [ ]:
import numba
print(numba.__version__)

In [13]:
import random
from numba import jit
@jit(nopython=True)
def monte_carlo_numba(nsamples):
    pi = 0.
    
   # Implement your code here
    cile_points=0
    square_points=0 
    for i in range(nsamples):
        x = random.random()
        y = random.random()
        d = x**2 + y**2
        if d<=1:
            cile_points += 1
    pi=4*(cile_points/nsamples)
    return pi

In [20]:
#time that monte carlo for 100000 points take in case of using python, cython and pythin with numba 
print("time that monte carlo take using only python is:")
%timeit monte_carlo_pi(100000)
print("time that monte carlo take using only cython is:")
%timeit monte_carlo_pi_cy(100000)
print("time that monte carlo take using python with numba is:")
%timeit monte_carlo_numba(100000)


time that monte carlo take using only python is:
67 ms ± 15.1 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
time that monte carlo take using only cython is:
8.19 ms ± 809 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
time that monte carlo take using python with numba is:
1.4 ms ± 480 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### Exercise 2 

In the [Introduction to Quantitative Economics](https://python.quantecon.org/intro.html) with Python lecture series you can learn all about finite-state Markov chains.

For now, let's just concentrate on simulating a very simple example of such a chain.

Suppose that the volatility of returns on an asset can be in one of two regimes — high or low.

The transition probabilities across states are as follows ![markov](../data/markov.png)

For example, let the period length be one day, and suppose the current state is high.

We see from the graph that the state tomorrow will be

- high with probability 0.8

- low with probability 0.2

Your task is to simulate a sequence of daily volatility states according to this rule.

Set the length of the sequence to `n = 1_000_000` and start in the high state.

Implement a pure Python version and a Numba version, and compare speeds.

To test your code, evaluate the fraction of time that the chain spends in the low state.

If your code is correct, it should be about 2/3.

Hints:

- Represent the low state as 0 and the high state as 1.

- If you want to store integers in a NumPy array and then apply JIT compilation, use `x = np.empty(n, dtype=np.int_)`.


In [32]:

def transition(state):
    if state==1:
        return np.random.choice([0,1],p=[0.2,0.8])
    else:
        return np.random.choice([0,1],p=[0.9,0.1])
def markov(n):
    state=1
    time=0
    for _ in range(n):
        state=transition(state)
        if state==0:
            time += 1
    return time/n

In [33]:
n=1000000

markov(n)

0.668091

as we remarque we get 2/3 